In [ ]:
import json
import pandas as pd
import random
import numpy as np
f = open('all_stock.json')
response = json.load(f)
stock = pd.DataFrame.from_dict(response)

f = open('demand.json')
response = json.load(f)
demand = pd.DataFrame.from_dict(response)

In [ ]:
MODEL = '45'
FAMILY = 'SHS'
# stock.columns
stock = stock[(stock.product_family == FAMILY) & (stock.product_model == MODEL) & (stock.virtual_location == 'GY3') & (stock.product_length < 5500)]
# stock = stock[(stock.product_family == FAMILY) & (stock.product_model == MODEL) & (stock.virtual_location == 'GY3') & (stock.product_length < 5500) & ("BLANL" not in stock.item_description.str)]
stock = stock.replace(np.nan, 0)
for index, r in enumerate(zip(*stock.to_dict("list").values())):                             
    if r[3] == 0:        
        stock.iloc[index, 3] = int(r[1][6:9]) / 10
        if "BLANK" not in r[0]:
            stock.iloc[index, 6] = 105.0
test = stock.groupby(["product_length", "product_g1"]).first()
test.reset_index(inplace=True)
test = test[test['product_pitch'] != 0]

In [ ]:
demand = demand[demand.facility.str.startswith('C0010')]
dem = demand[(demand.product_family == FAMILY) & (demand.product_model == MODEL) & (~demand.item_description.str.contains('(GP)')) & (demand.product_length < 5500)].copy()
dem_group = dem.groupby('product_length').first()
dem_group['qty'] = dem.groupby('product_length')['order_quantity'].sum()
dem_group = dem_group.reset_index()
dem_group.sort_values('qty', inplace=True)
dem_group.head(50)

In [ ]:
dem_r = dem_group[::-1]
dem_len_list = np.array(dem_r.product_length.tolist(), int)
dem_types = len(dem_len_list)
dem_qty_list = dem_r['qty'].tolist()
dem_g_list = dem_r['product_g1'].tolist()
stock_g_list = test['product_g1'].tolist()
clamp = 10
kerf = 2

def get_g2(length, g1, pitch):
    return (length - g1) % pitch    

def get_possible_quantity(long_len, long_g1, short_len, short_g1, pitch):
    possible_quantity = 0
    loss_at_cut = [0]
    len_at_point = [long_len]
    # while long_len > short_len + pitch:
    while (((short_g1 + clamp + kerf <= long_g1) & ((long_len - long_g1) >= (short_len - short_g1))) | ((long_len - long_g1) > short_len)):
        cut = 1
        front_loss = 0
        if long_g1 == short_g1:            
            cut = 0
        elif short_g1 + clamp + kerf < long_g1:
            front_loss = long_g1 - short_g1                        
        else:
            front_loss = long_g1  + (pitch - short_g1)
        short_g2 = get_g2(short_len, short_g1, pitch) 
        long_g1 = pitch - (short_g2 + kerf)
        possible_quantity += 1
        long_len -= short_len + front_loss + kerf        
        len_at_point.append(long_len)
        loss_at_cut.append(front_loss + kerf * cut) 
    return possible_quantity, len_at_point, loss_at_cut

# def numbers_with_sum(stock_types, stock_len_list, dem_len, dem_qty):
def generate_arr(stock_types, stock_len_list, r, d_index, longest):
    dem_len = r[0]
    dem_qty = r[-1]
    pitch = r[9]
    arr = np.zeros(stock_types, dtype=int)    
    loss = np.zeros(stock_types, dtype=int)    
    if dem_qty == 0:
        return arr, loss    
    for index, stock_len in enumerate(stock_len_list):        
        possible_quantity, len_at_point, loss_at_cut = get_possible_quantity(stock_len, stock_g_list[index], dem_len, dem_g_list[d_index], pitch)        
        if possible_quantity:
            rand_val = random.randint(0 + longest, min(possible_quantity, dem_qty))
            stock_len_list[index] = len_at_point[rand_val]
            arr[index] = rand_val
            loss[index] = sum(loss_at_cut[:(rand_val+1)])
            dem_qty -= rand_val
            # print(stock_len, index, dem_len, loss)
            if dem_qty == 0:
                return arr, loss
        else:
            pass
    return arr, loss

In [22]:
total_sum = 0.0
dem_qty_list = dem_r['qty'].tolist()
count_ = np.count_nonzero(dem_qty_list)
stock_len_list_original = np.array(test.product_length.tolist(), int)
stock_types = len(stock_len_list_original)
dem_repeat = np.repeat([dem_len_list], stock_types, axis=0).T
while count_:
# for i in range(10):
    stock_len_list = stock_len_list_original.copy()
    best = []
    best_loss = []
    min_ = 10000
    best_stock_len_list = []
    best_remainder = []
    longest = False    
    for i in range(count_ * 30):        
        stock_len_list = np.array(test.product_length.tolist(), int)
        arr = np.zeros((0,stock_types), dtype=int)
        loss = np.zeros((0,stock_types), dtype=int)
        for index, r in enumerate(zip(*dem_r.to_dict("list").values())):                             
            if ~longest & int(r[-1]):
                longest = True
            arr_, loss_ = generate_arr(stock_types, stock_len_list, r, index, longest)
            arr = np.append(arr, np.array([arr_]), axis=0)
            loss = np.append(loss, np.array([loss_]), axis=0)            
        remainder = (stock_len_list != stock_len_list_original) * stock_len_list
        cut_loss = loss.sum()
        total_loss = cut_loss + sum(remainder)        
        if total_loss < min_:
            best = arr
            min_ = total_loss
            best_stock_len_list = stock_len_list
            best_loss = np.append(loss, np.array([remainder]), axis=0)                        
    stock_i = np.argmin(best_loss.sum(axis = 0))
    dem_r['qty'] = dem_qty_list - best.sum(axis=1)    
    columns = np.array(test.product_length.tolist(), int)
    indexes = dem_r['product_length'].tolist()
    indexes.append("lost")
    df = pd.DataFrame(best, columns = columns, index=dem_r['product_length'].tolist())
    df['qty'] = dem_r.qty.tolist()
    dem_qty_list = dem_r['qty'].tolist()    
    count_ = np.count_nonzero(dem_qty_list)
    redunduncy = np.count_nonzero(best !=0, axis=1)        
    print(df)
    

        2020  3020  4050  5050   qty
2310.0     0     0     0     0   0.0
2286.0     0     1     1     0   0.0
4427.0     0     0     0     1  21.0
3448.0     0     0     0     0  20.0
1470.0     0     0     0     0   0.0
990.0      0     0     0     0   0.0
1218.0     0     0     0     0   0.0
920.0      0     0     0     0   0.0
4875.0     0     0     0     0   2.0
1830.0     0     0     0     0   0.0
885.0      0     0     0     0   0.0
940.0      0     0     0     0   0.0
2150.0     0     0     0     0   2.0
2050.0     0     0     0     0   2.0
4851.0     0     0     0     0   2.0
2585.0     0     0     0     0   2.0
2670.0     0     0     0     0   2.0
2880.0     0     0     0     0   2.0
178.0      0     0     0     0   0.0
3549.0     0     0     0     0   1.0
        2020  3020  4050  5050   qty
2310.0     0     0     0     0   0.0
2286.0     0     0     0     0   0.0
4427.0     0     0     0     1  20.0
3448.0     0     0     1     0  19.0
1470.0     0     0     0     0   0.0
9

In [21]:
testarr = np.array([39, 710, 750, 430])
np.argmin(testarr)


0

In [ ]:
(np.array(test.product_length.tolist(), int) - (arr * dem).sum(axis=0)).sum()

In [ ]:
df = pd.DataFrame(best * dem, columns = test.product_length.tolist())
df.index = dem_len_list
df['qty'] = dem_r.qty.tolist()
df_sum = df.sum()
arr2 = np.array(test.product_length.tolist(), int) - df_sum.values[:-1]
arr2 = np.append(arr2, 156)
df.append(pd.DataFrame(arr2.reshape(1,-1), index=["Loss"], columns=list(df)))



In [ ]:
dem_len_list = np.array(dem_group.product_length.tolist(), int)
stock_len_list = np.array(test.product_length.tolist(), int)

def numbers_with_sum(stock_index, length, n, k):    
    s = stock_len_list[stock_index]
    if k > 0:
        if s < length:
            num = 0            
        else:        
            num = random.randint(0, s // length)
    else:
        num = 0
    if n == 1:
        return [k]    
    return [num] + numbers_with_sum(stock_index+1, length, n - 1, k - num)

In [ ]:
test

In [ ]:
# arr = np.empty((0,len(dem_len_list)), int)
print(stock_len_list)
for d in dem_len_list:
    print(d, [s % d for s in stock_len_list])
    # arr = np.append(arr, np.array([r % x for x in dem_len_list]), axis=0)
arr

In [ ]:
dem_r = dem_group[::-1]
dem_len_list = np.array(dem_r.product_length.tolist(), int)
dem_types = len(dem_len_list)
dem_qty_list = dem_r['qty'].tolist()

In [ ]:
total_sum = 0.0
for f in range(50):
    best = []
    min_ = 10000
    loss_sum_min = []
    dem_qty_list = dem_r['qty'].tolist()        
    count_ = np.count_nonzero(dem_qty_list)
    print(count_)
    for i in range(count_ * 100):
        stock_len_list = np.array(test.product_length.tolist(), int)
        stock_types = len(stock_len_list)
        dem = np.repeat([dem_len_list], 4, axis=0).T
        arr = np.zeros((0,stock_types), dtype=int)

        def numbers_with_sum(dem_index, dem_len, dem_qty):
            arr = np.zeros(stock_types, dtype=int)
            for index, stock_len in enumerate(stock_len_list):
                possible_quantity = stock_len // dem_len
                if possible_quantity:
                    # if stock_len_list[index + 1] & 1:
                    #     pass
                    rand_val = random.randint(0, min(possible_quantity, dem_qty))
                    stock_len_list[index] -= rand_val * dem_len
                    arr[index] = rand_val
                    dem_qty -= rand_val
                    if dem_qty == 0:
                        return arr                    
                else:
                    pass        
            return arr                
        for index, r in enumerate(zip(*dem_r.to_dict("list").values())):                             
            arr = np.append(arr, np.array([numbers_with_sum(index, r[0], r[-1])]), axis=0)        
        loss_sum = np.array(test.product_length.tolist(), int) - (arr * dem).sum(axis=0)
        cur_loss = (loss_sum).sum()
        # cur_sum = (arr * dem).sum()    
        if cur_loss < min_:
            best = arr
            min_ = cur_loss    
    dem_r['qty'] = dem_qty_list - best.sum(axis=1)
    df = pd.DataFrame(best * dem, columns = test.product_length.tolist())
    df.index = dem_len_list
    df['qty'] = dem_r.qty.tolist()
    df_sum = df.sum()
    arr2 = np.array(test.product_length.tolist(), int) - df_sum.values[:-1]
    total_sum += arr2.sum()
    arr2 = np.append(arr2, 156)
    pd.concat([df, pd.DataFrame(arr2.reshape(1,-1), index=["Loss"], columns=list(df))])

    # df.append(pd.DataFrame(arr2.reshape(1,-1), index=["Loss"], columns=list(df)))    
    print(df)
    # print(total_sum)